In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os

In [ ]:
class SensorDataset(Dataset):
    # For .csv ingestion
    def __init__(self, csv_dir, labels_dict):
        self.csv_dir = csv_dir
        self.file_list = os.listdir(csv_dir)
        self.labels_dict = labels_dict

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        file_path = os.path.join(self.csv_dir, file_name)
        
        df = pd.read_csv(file_path)
        
        features = df.iloc[:, 1:].values
        features = torch.tensor(features, dtype=torch.float32)
        
        label = self.labels_dict[file_name]
        label = torch.tensor(label, dtype=torch.float32)
        
        return features, label

def create_dataloader(csv_dir, labels_dict, batch_size=4):
    # Input directory of samples (in .csv) and labels ({"file1.csv": 0, ...})
    dataset = SensorDataset(csv_dir, labels_dict)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        return out